In [18]:
import os 
import sys
from tqdm import tqdm
import importlib
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [19]:
BATCH_SIZE = 256
T = 20
VOCAB = ['start', 'view', 'click', 'install']
EVENT_VOCAB_DIM = len(VOCAB)
EMB_DIM = 16
HIDDEN_DIM = 128

## Classifier Model

In [20]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, LSTM, Embedding, Reshape, Dense, Dropout, Activation, Multiply, Add, Lambda
from tensorflow.keras import regularizers

from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall


def build_classifier(batch_size, T, event_vocab_dim, emb_dim, hidden_dim, dropout_rate=0.25):
    # normal LSTM
    i_et = Input(batch_shape=(batch_size, None, event_vocab_dim), name='event_type')  # input of discrete feature event type
    i_ts = Input(batch_shape=(batch_size, None, 1), name='time_delta_in')  # input of continuous feature timestamp
    
    embed0 = Dense(emb_dim, name='dense_emb')(i_et) # dense matrix size: 6*16
    merged0 = tf.concat([embed0, i_ts], axis=2)
    
    hm = LSTM(hidden_dim,
          name='lstm_token',
          stateful=False,
          return_sequences=False, 
          kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(merged0)

    tm = LSTM(hidden_dim,
          name='lstm_time',
          stateful=False,
          return_sequences=False,
          kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(merged0)
    
    token_time_comb = tf.keras.layers.concatenate([hm, tm], axis=1)
    
    dropped = Dropout(rate=dropout_rate)(token_time_comb)
    
    prob = Dense(1, 
             activation='sigmoid',
             name='final',
             kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
             bias_initializer=tf.keras.initializers.Constant(value=0.1))(dropped)
        
    classifier = Model(
        inputs=[i_et, i_ts],
        outputs=prob)
        
    metrics = [
        BinaryAccuracy(name='accuracy'),
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(num_thresholds=500, curve='PR', name='auc_pr'),
        AUC(num_thresholds=500, curve='ROC', name='auc_roc')
    ]
    
    classifier.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(lr=0.001),
        metrics=metrics)
    
    return classifier

In [21]:
classifier = build_classifier(batch_size=BATCH_SIZE,
                              T=T,
                              event_vocab_dim=EVENT_VOCAB_DIM,
                              emb_dim=EMB_DIM,
                              hidden_dim=HIDDEN_DIM,    
                              dropout_rate=0.25)

In [22]:
classifier.summary()

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
event_type (InputLayer)         [(256, None, 4)]     0                                            
__________________________________________________________________________________________________
dense_emb (Dense)               (256, None, 16)      80          event_type[0][0]                 
__________________________________________________________________________________________________
time_delta_in (InputLayer)      [(256, None, 1)]     0                                            
__________________________________________________________________________________________________
tf.concat_17 (TFOpLambda)       (256, None, 17)      0           dense_emb[0][0]                  
                                                                 time_delta_in[0][0]       

## Load Saved TF Datasets

In [23]:
path_prefix = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/data/classifier_tf_datasets'

In [40]:
train_names_500 = [
'1-to-500-im',
'1-to-500-up',
'1-to-500-gs-G0',
'1-to-500-gs-G1',
'1-to-500-gs-G2',
'1-to-500-mcc-G2',
'1-to-500-mcc-G1'
]

val_data_path_500 = os.path.join(path_prefix, '1-to-500-standalone-val.tf.data')

In [41]:
results_save_path_prefix = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/experiments'

In [42]:
# For each dataset, run 10 trails; At each Trail, build a new model and train to get best metrics

TRIALS = 3

epochs = 20
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3
)

In [46]:
from tensorflow import TensorSpec
element_spec = ((TensorSpec(shape=(BATCH_SIZE, T, 4), dtype=tf.float64, name=None), 
                 TensorSpec(shape=(BATCH_SIZE, T, 1), dtype=tf.float64, name=None)),
                TensorSpec(shape=(BATCH_SIZE, 1), dtype=tf.float64, name=None))

## Experiment for 1-to-ratio

In [47]:
val_dataset_500 = tf.data.experimental.load(val_data_path_500, element_spec)

In [48]:
for name in train_names_500:
    dataset_path = os.path.join(path_prefix, name + '.tf.data')
    train_dataset = tf.data.experimental.load(dataset_path, element_spec)
    
    print('Processing Data:', dataset_path)
    all_run_bests = []
    
    for run in range(TRIALS):        
        print('Trial:', run)
        
        classifier_run = build_classifier(batch_size=BATCH_SIZE,
                                          T=T,
                                          event_vocab_dim=EVENT_VOCAB_DIM,
                                          emb_dim=EMB_DIM,
                                          hidden_dim=HIDDEN_DIM,    
                                          dropout_rate=0.25)
        
        run_results = classifier_run.fit(train_dataset, 
                                        epochs=epochs,
                                        validation_data=val_dataset_500,
                                        shuffle=True,
                                        callbacks=[early_stopping])
        
        run_history = run_results.history
        best_ind = np.argmin(run_history['val_loss'])
        run_best = {k : v_list[best_ind] for k, v_list in run_history.items()}
        
        all_run_bests.append(run_best)
    
    df_all_bests = pd.DataFrame(all_run_bests)
    results_save_path = os.path.join(results_save_path_prefix, name + '_all_bests.csv')
    df_all_bests.to_csv(results_save_path)

Processing Data: /home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/data/classifier_tf_datasets/1-to-500-im.tf.data
Trial: 0
Epoch 1/20
782/782 [==============================] - 71s 87ms/step - loss: 0.0316 - accuracy: 0.9891 - precision: 0.0040 - recall: 0.0154 - auc_pr: 0.0025 - auc_roc: 0.5410 - val_loss: 0.0124 - val_accuracy: 0.9980 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc_pr: 0.0432 - val_auc_roc: 0.7603
Epoch 2/20
782/782 [==============================] - 66s 84ms/step - loss: 0.0117 - accuracy: 0.9980 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc_pr: 0.0242 - auc_roc: 0.8460 - val_loss: 0.0098 - val_accuracy: 0.9980 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc_pr: 0.2349 - val_auc_roc: 0.8925
Epoch 3/20
782/782 [==============================] - 65s 84ms/step - loss: 0.0099 - accuracy: 0.9980 - precision: 0.2528 - recall: 0.0048 - auc_pr: 0.0969 - auc_roc: 0.9179 - val_loss: 0.0086 - val_accuracy: 0.9982 - val_pr

782/782 [==============================] - 65s 84ms/step - loss: 0.0033 - accuracy: 0.9989 - precision: 0.8477 - recall: 0.5599 - auc_pr: 0.7684 - auc_roc: 0.9919 - val_loss: 0.0054 - val_accuracy: 0.9987 - val_precision: 0.8170 - val_recall: 0.4800 - val_auc_pr: 0.6556 - val_auc_roc: 0.9372
Epoch 14/20
782/782 [==============================] - 65s 83ms/step - loss: 0.0030 - accuracy: 0.9991 - precision: 0.8792 - recall: 0.6229 - auc_pr: 0.8039 - auc_roc: 0.9844 - val_loss: 0.0052 - val_accuracy: 0.9987 - val_precision: 0.7667 - val_recall: 0.5175 - val_auc_pr: 0.6637 - val_auc_roc: 0.9436
Trial: 2
Epoch 1/20
782/782 [==============================] - 69s 86ms/step - loss: 0.0272 - accuracy: 0.9960 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc_pr: 0.0035 - auc_roc: 0.5963 - val_loss: 0.0115 - val_accuracy: 0.9980 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc_pr: 0.0903 - val_auc_roc: 0.8128
Epoch 2/20
782/782 [==============================] - 67s 85ms/step - los

Epoch 12/20
782/782 [==============================] - 66s 85ms/step - loss: 0.0039 - accuracy: 0.9987 - precision: 0.7898 - recall: 0.4579 - auc_pr: 0.6989 - auc_roc: 0.9862 - val_loss: 0.0043 - val_accuracy: 0.9989 - val_precision: 0.8502 - val_recall: 0.5250 - val_auc_pr: 0.7227 - val_auc_roc: 0.9562
Epoch 13/20
782/782 [==============================] - 66s 85ms/step - loss: 0.0033 - accuracy: 0.9989 - precision: 0.8467 - recall: 0.5683 - auc_pr: 0.7720 - auc_roc: 0.9900 - val_loss: 0.0043 - val_accuracy: 0.9988 - val_precision: 0.7468 - val_recall: 0.5900 - val_auc_pr: 0.7106 - val_auc_roc: 0.9659
Epoch 14/20
782/782 [==============================] - 67s 85ms/step - loss: 0.0031 - accuracy: 0.9990 - precision: 0.8573 - recall: 0.5986 - auc_pr: 0.7808 - auc_roc: 0.9952 - val_loss: 0.0044 - val_accuracy: 0.9988 - val_precision: 0.8090 - val_recall: 0.5400 - val_auc_pr: 0.6925 - val_auc_roc: 0.9633
Epoch 15/20
782/782 [==============================] - 66s 85ms/step - loss: 0.0028 -

1562/1562 [==============================] - 112s 72ms/step - loss: 0.0017 - accuracy: 0.9995 - precision: 0.9993 - recall: 0.9998 - auc_pr: 0.9999 - auc_roc: 1.0000 - val_loss: 0.0103 - val_accuracy: 0.9988 - val_precision: 0.7500 - val_recall: 0.5700 - val_auc_pr: 0.5656 - val_auc_roc: 0.8483
Epoch 11/20
1562/1562 [==============================] - 112s 72ms/step - loss: 7.8427e-04 - accuracy: 0.9998 - precision: 0.9997 - recall: 0.9999 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0091 - val_accuracy: 0.9985 - val_precision: 0.6274 - val_recall: 0.5725 - val_auc_pr: 0.5571 - val_auc_roc: 0.8853
Epoch 12/20
1562/1562 [==============================] - 112s 72ms/step - loss: 0.0010 - accuracy: 0.9997 - precision: 0.9996 - recall: 0.9999 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0104 - val_accuracy: 0.9986 - val_precision: 0.7276 - val_recall: 0.4675 - val_auc_pr: 0.5477 - val_auc_roc: 0.8419
Epoch 13/20
1562/1562 [==============================] - 113s 72ms/step - loss: 4.51

1562/1562 [==============================] - 112s 72ms/step - loss: 0.0091 - accuracy: 0.9976 - precision: 0.9958 - recall: 0.9994 - auc_pr: 0.9994 - auc_roc: 0.9997 - val_loss: 0.0147 - val_accuracy: 0.9957 - val_precision: 0.2960 - val_recall: 0.8400 - val_auc_pr: 0.5751 - val_auc_roc: 0.9693
Epoch 4/20
1562/1562 [==============================] - 112s 72ms/step - loss: 0.0059 - accuracy: 0.9985 - precision: 0.9975 - recall: 0.9996 - auc_pr: 0.9997 - auc_roc: 0.9998 - val_loss: 0.0102 - val_accuracy: 0.9975 - val_precision: 0.4288 - val_recall: 0.7900 - val_auc_pr: 0.6465 - val_auc_roc: 0.9474
Epoch 5/20
1562/1562 [==============================] - 112s 72ms/step - loss: 0.0045 - accuracy: 0.9989 - precision: 0.9980 - recall: 0.9997 - auc_pr: 0.9997 - auc_roc: 0.9999 - val_loss: 0.0102 - val_accuracy: 0.9974 - val_precision: 0.4225 - val_recall: 0.7900 - val_auc_pr: 0.6668 - val_auc_roc: 0.9450
Trial: 4
Epoch 1/20
1562/1562 [==============================] - 118s 74ms/step - loss: 0.

...

1562/1562 [==============================] - 111s 71ms/step - loss: 0.0018 - accuracy: 0.9994 - precision: 0.9997 - recall: 0.9990 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0045 - val_accuracy: 0.9989 - val_precision: 0.8632 - val_recall: 0.5050 - val_auc_pr: 0.6965 - val_auc_roc: 0.9733
Epoch 10/20
1562/1562 [==============================] - 110s 71ms/step - loss: 0.0017 - accuracy: 0.9994 - precision: 0.9998 - recall: 0.9991 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0042 - val_accuracy: 0.9988 - val_precision: 0.8500 - val_recall: 0.5100 - val_auc_pr: 0.7121 - val_auc_roc: 0.9657
Epoch 11/20
1562/1562 [==============================] - 110s 70ms/step - loss: 0.0015 - accuracy: 0.9995 - precision: 0.9997 - recall: 0.9992 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0056 - val_accuracy: 0.9987 - val_precision: 0.9068 - val_recall: 0.3650 - val_auc_pr: 0.6399 - val_auc_roc: 0.9266
Epoch 12/20
1562/1562 [==============================] - 110s 70ms/step - loss: 0.0014 -

Epoch 11/20
1562/1562 [==============================] - 110s 71ms/step - loss: 0.0016 - accuracy: 0.9995 - precision: 0.9998 - recall: 0.9992 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0042 - val_accuracy: 0.9989 - val_precision: 0.8065 - val_recall: 0.5625 - val_auc_pr: 0.7075 - val_auc_roc: 0.9658
Trial: 3
Epoch 1/20
1562/1562 [==============================] - 113s 71ms/step - loss: 0.0305 - accuracy: 0.9907 - precision: 0.9902 - recall: 0.9915 - auc_pr: 0.9981 - auc_roc: 0.9979 - val_loss: 0.0104 - val_accuracy: 0.9980 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc_pr: 0.1149 - val_auc_roc: 0.9494
Epoch 2/20
1562/1562 [==============================] - 109s 70ms/step - loss: 0.0053 - accuracy: 0.9989 - precision: 0.9999 - recall: 0.9979 - auc_pr: 0.9999 - auc_roc: 0.9998 - val_loss: 0.0096 - val_accuracy: 0.9980 - val_precision: 1.0000 - val_recall: 0.0050 - val_auc_pr: 0.2313 - val_auc_roc: 0.9707
Epoch 3/20
1562/1562 [==============================] - 110s 

Epoch 12/20
1562/1562 [==============================] - 111s 71ms/step - loss: 0.0012 - accuracy: 0.9996 - precision: 0.9998 - recall: 0.9994 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0035 - val_accuracy: 0.9990 - val_precision: 0.7552 - val_recall: 0.7250 - val_auc_pr: 0.7859 - val_auc_roc: 0.9777
Epoch 13/20
1562/1562 [==============================] - 110s 71ms/step - loss: 0.0011 - accuracy: 0.9996 - precision: 0.9998 - recall: 0.9995 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0046 - val_accuracy: 0.9989 - val_precision: 0.8767 - val_recall: 0.4975 - val_auc_pr: 0.7285 - val_auc_roc: 0.9416
Epoch 14/20
1562/1562 [==============================] - 110s 71ms/step - loss: 0.0011 - accuracy: 0.9996 - precision: 0.9998 - recall: 0.9995 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0040 - val_accuracy: 0.9988 - val_precision: 0.6897 - val_recall: 0.7500 - val_auc_pr: 0.7800 - val_auc_roc: 0.9715
Processing Data: /home/lun/project-basileus/multitype-sequence-generation-by

1562/1562 [==============================] - 110s 71ms/step - loss: 9.9576e-04 - accuracy: 0.9996 - precision: 0.9998 - recall: 0.9995 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0044 - val_accuracy: 0.9990 - val_precision: 0.9561 - val_recall: 0.5450 - val_auc_pr: 0.8048 - val_auc_roc: 0.9409
Trial: 2
Epoch 1/20
1562/1562 [==============================] - 114s 71ms/step - loss: 0.0646 - accuracy: 0.9712 - precision: 0.9719 - recall: 0.9709 - auc_pr: 0.9911 - auc_roc: 0.9909 - val_loss: 0.0097 - val_accuracy: 0.9981 - val_precision: 0.5641 - val_recall: 0.1100 - val_auc_pr: 0.1546 - val_auc_roc: 0.9122
Epoch 2/20
1562/1562 [==============================] - 111s 71ms/step - loss: 0.0055 - accuracy: 0.9990 - precision: 0.9998 - recall: 0.9981 - auc_pr: 0.9998 - auc_roc: 0.9998 - val_loss: 0.0078 - val_accuracy: 0.9982 - val_precision: 0.6460 - val_recall: 0.1825 - val_auc_pr: 0.3255 - val_auc_roc: 0.9218
Epoch 3/20
1562/1562 [==============================] - 111s 71ms/step - loss

Epoch 11/20
1562/1562 [==============================] - 111s 71ms/step - loss: 0.0014 - accuracy: 0.9995 - precision: 0.9998 - recall: 0.9993 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0028 - val_accuracy: 0.9992 - val_precision: 0.8080 - val_recall: 0.7575 - val_auc_pr: 0.8449 - val_auc_roc: 0.9865
Epoch 12/20
1562/1562 [==============================] - 111s 71ms/step - loss: 0.0011 - accuracy: 0.9996 - precision: 0.9998 - recall: 0.9994 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0026 - val_accuracy: 0.9993 - val_precision: 0.8892 - val_recall: 0.7225 - val_auc_pr: 0.8578 - val_auc_roc: 0.9782
Epoch 13/20
1562/1562 [==============================] - 111s 71ms/step - loss: 0.0010 - accuracy: 0.9997 - precision: 0.9998 - recall: 0.9995 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0029 - val_accuracy: 0.9992 - val_precision: 0.9231 - val_recall: 0.6600 - val_auc_pr: 0.8498 - val_auc_roc: 0.9757
Epoch 14/20
1562/1562 [==============================] - 111s 71ms/step - lo